This code was written to generate the data in the paper "Ekedahl-Oort Types and Newton Polygons of Abelian Covers
of P1 Branched at Three Points" by Darren Schmidt. Some of the code was taken from https://github.com/jeremybooher/supersingular-3pointed-covers, by Jeremy Booher and Rachel Pries, used and modified with permission. We enumerate every abelian cover of P^1 branched at three points with a given genus, and then compute the natural density of the primes such that the curves are supersingular, superspecial, and where the Newton polygons and Ekedahl-Oort types are unlikely.

In [ ]:
from functools import cmp_to_key

In [4]:
def _partial_enumeration(genus_bound,d,e,f):
    """Enumerate possible (d,e,f),r,s given (d,e,f).  Used in enumerate_covers"""
    new_stuff = []
    r = 1
    while r * e * f - (1 + f/d + e/d) <= 2 * genus_bound -2:
        if r * e * f - (1 + f/d + e/d) <=0 : #won't have genus > 1
            r = r+1
            continue
        s = 1
        while  s * ( r * e * f - (1 + f/d + e/d)) <= 2 * genus_bound - 2:
            #is genus an integer?  is s| rd satisfied? are d,e,f pairwise relatively prime
            g = s * ( r * e * f - (1 + f/d + e/d)) /2 + 1
            if g in ZZ and r * d /s in ZZ and gcd(d,e) == 1 and gcd(d,f) == 1 and gcd(e,f) ==1: 
                new_stuff.append([[d,e,f],r,s])
            s = s+1
        r = r+1

    return new_stuff


def enumerate_covers(genus_bound):
    """Find all abelian covers of the projective line with three points of ramification and 1<genus <= genus_bound.
    Note that requiring genus>1 is fine and removes an infinite family of boring examples of covers
    
    Returns a list of (d,e,f), s ,r using Ekedahl's notation.
    
    Note: d,e,f are e_{i,j} in our notation"""

    candidates = []
    #remember that d<=e<=f by convention
    f = 1
    
    while f <= 2*genus_bound + 2:
        e = 1
        
        while e <= f:
            done_e = true
            d = 1 
            while d <= e and e * f - (1 + f/d + e/d) <= 2 * genus_bound-2:
                new_stuff = _partial_enumeration(genus_bound,d,e,f)
                candidates.extend(new_stuff)
                d = d +1
                
            e = e +1        
        
        f = f +1
    return candidates

In [5]:
def convert_cover(triple):
    """Convert to the (a,b,c,s) notation from the (d,e,f), s ,r (both used by Ekedahl).
    a,b,c are the orders of the generators of inertia, s is the index of last inertia in G"""
    d,e,f = expand( triple[0])
    r = triple[1]
    s = triple[2]

    a = r * d * e
    b = r * d * f
    c = r * e * f
    genus = s * ( r * e * f - (1 + f/d + e/d)) /2 + 1
    
    return [a,b,c,s],genus

In [6]:
def cyclic_inertial_generators(a,b,c,s):
    """Assuming the cover has s=1, find possible generators for inertia with given (a,b,c,s).
    Prune the list to account for isomorphic covers"""


    
    inert_gens = [] #inertial generators to return
    isomorphic_gens =[] #cases isomorphic to ones in inert_gens
    
    # cyclic Galois group is Z/(c)
    
    order = c*s
    G = Integers(order)
    #up to automorphism, may as well assume generator for last inertia group is 1
    third = G(1)
  
    #as generators of inertia sum to 0, look at possible values for the first and see which are valid:
    #first has order a and second has order b

    for g in G:
        if g.order() == a and (-third - g ).order() == b:
            first = g
            second = -third - g

            data = [first.lift(),second.lift(),third.lift()]

            if data not in isomorphic_gens:
                #it's new
                inert_gens.append(data)
                isomorphic_gens.append(data)

                #if some of a,b,c are equal, may permuate and scale to get an isomorphic cover.  Only write down one of them
                #in particular, if a=b!= c could swap first and second inertial elements.
                #if c = b != a, could swap and rescale
                #if c= b= = a, could use S_3 permutation and rescale
                if a == b and b != c:
                    isomorphic_gens.append([second.lift(),first.lift(),third.lift()])
                elif c == b and b != a:
                    scaling = second^(-1)
                    isomorphic_gens.append([(first*scaling).lift(),scaling.lift(),1])
                elif c == b and b == a:
                    isomorphic_gens.append([second.lift(),first.lift(),third.lift()])
                    isomorphic_gens.append([first^(-1) * second,first^(-1),1])
                    isomorphic_gens.append([first^(-1),first^(-1) * second,1])
                    isomorphic_gens.append([second^(-1),second^(-1) * first,1])
                    isomorphic_gens.append([second^(-1) * first,second^(-1),1])
                    
    return inert_gens,order

def alternate_inertial_generators(a,b,c,s):
    """The cover is a product Z/(i) \times Z/(j) with j|i and ij = cs,
    find possible generators for inertia (up to automorphism) with given (a,b,c,s)"""

    inert_gen_list = []
    order = c * s
    
    # Galois group is Z/(c) \times Z/(s) 
    if s.divides(a) and a.divides(b) and b.divides(c):
        
        
        
        G = AdditiveAbelianGroup([c,s])
        inert_gens = [G]
        #up to automorphism, may as well assume generator for last inertia group is (1,0) as it has order c
        third = G([1,0])
        c0 = [1,0]
        #the other two generators cannot lie in <(1,0)> as they together generate the whole group.

        #as generators of inertia sum to 0, look at possible values for the first and see which are valid:
        #first has order a and second has order b
        
        R1 = Integers(c)
        R2 = Integers(s)

        for i in range(c):

            for j in range(s):
                
                secondElement = [i,j]
                thirdElement = [R1(-1-i).lift(),R2(-j).lift()]
                secondOrder = lcm(c/gcd(i,c),s/gcd(j,s))
                thirdOrder = lcm(c/gcd(c,thirdElement[0]),s/gcd(s,thirdElement[1]))
                if secondOrder == a and thirdOrder == b:
                    if G.submodule([third, G(secondElement)]).order() == c*s:
                        inert_gens.append([secondElement,thirdElement,c0])
                        
        inert_gen_list.append(inert_gens)

    #Not guaranteed to be Z/(c) \times Z/(s), loops through all possible groups that it could be
    else:
        
        lcmGroup = lcm(a,lcm(b,c))
        for i in range(1, floor(order/lcmGroup)+1):
            #Constructs group of size Z/(j) \times Z/(k) with k | j and kj = cs
            j = i * lcmGroup
            if j.divides(order):
                k = Integer(order/j)
                if k.divides(j):
                    R1 = Integers(j)
                    R2 = Integers(k)
                    G = AdditiveAbelianGroup([j,k])
                    inert_gens = [G]
                    
                    #Need three elements of G with order a, b, and c such that their sum is 0 and any two of them generate G.
                    #If j == c, can choose (1,0) in G as third element.
                    if j == c:
                        third = G([1,0])
                        c0 = [1,0]
                        
                        #If k == a, can choose (0,1) as first element. Then find possible second element with order b.
                        if k == a:
                        
                            first = G([0,1])
                            secondElement = [0,1]

                            thirdElement = [R1(-c0[0]).lift(), R2(-c0[1]-1).lift()]
                            thirdOrder = lcm(j/gcd(j,thirdElement[0]),k/gcd(k,thirdElement[1]))

                            if thirdOrder == b:
                                inert_gens.append([secondElement,thirdElement,c0])

                            inert_gen_list.append(inert_gens)
                    
                
                        else:
                        #Otherwise, need to find any other generators that work
                            for m in range(j):
                                for n in range(k):
                                
                                    secondElement = [m,n]
                                    thirdElement = [R1(-c0[0]-m).lift(),R2(-c0[1]-n).lift()]
                                    secondOrder = lcm(j/gcd(m,j),k/gcd(n,k))
                                    thirdOrder = lcm(j/gcd(j,thirdElement[0]),k/gcd(k,thirdElement[1]))
                                    if secondOrder == a and thirdOrder == b:
                                        if G.submodule([third, G(secondElement)]).order() == c*s:
                                            inert_gens.append([secondElement,thirdElement,c0])
                            inert_gen_list.append(inert_gens)
                        
                    else:
                        #Need to loop through all possible generators for G with needed properties
                        for m in range(j):
                            for n in range(k):
                                if lcm(j/gcd(j,m),k/gcd(k,n)) == c:
                                    c0 = [m,n]
                                    third = G(c0)
                                    
                                    
                                    if k == a:
                        
                                        first = G([0,1])
                                        secondElement = [0,1]

                                        thirdElement = [R1(-c0[0]).lift(), R2(-c0[1]-1).lift()]
                                        thirdOrder = lcm(j/gcd(j,thirdElement[0]),k/gcd(k,thirdElement[1]))

                                        if thirdOrder == b:
                                            inert_gens.append([secondElement,thirdElement,c0])

                                        inert_gen_list.append(inert_gens)
                                        
                                    else:
                                        for m in range(j):
                                            for n in range(k):
                                
                                                secondElement = [m,n]
                                                thirdElement = [R1(-c0[0]-m).lift(),R2(-c0[1]-n).lift()]
                                                secondOrder = lcm(j/gcd(m,j),k/gcd(n,k))
                                                thirdOrder = lcm(j/gcd(j,thirdElement[0]),k/gcd(k,thirdElement[1]))
                                                if secondOrder == a and thirdOrder == b:
                                                    if G.submodule([third, G(secondElement)]).order() == c*s:
                                                        inert_gens.append([secondElement,thirdElement,c0])
                  
                                        inert_gen_list.append(inert_gens)
                        
                 
        
                
    
    return inert_gen_list

In [7]:
def present_by_genus(min,max):
    """Create dictionaries of all covers where min <= genus <= max, indexed by genus"""
    data  = enumerate_covers(max)

    collected_covers = {} #(a,b,c,s)
    for n in range(min,max+1):
        collected_covers[n] = []

    for triple in data:
        data,genus = convert_cover(triple)
        if min <= genus and genus <= max:
            collected_covers[genus].append(data)

    return collected_covers

In [8]:
#auxiliary functions
def compute_fj(j,a1,a2,a3,m):
    return -1 + frac(-1 * j * a1/m) + frac(-1 * j * a2/m) + frac(-1 * j * a3/m)

def compute_dj(j,m):
    return ((Integers(m))(j)).order() #it is additive order

In [9]:
def compare_words(a,b):
    """Lexicographic comparison of words in F and V of same length with F < V"""
    #Returns 1 if a < b and -1 if a > b
    for i in range(len(a)):
        if a[i] == "F" and b[i] == "V":
            return 1
        if a[i] == "V" and b[i] == "F":
            return -1
        
    return 0

In [10]:
def all_cycles(w,k):
    """Given a word w, left shifts the word k-times"""
    return w[k:] + w[:k]

In [11]:
def canonical_type(W):
    """Computes the canonical type, a precursor to computing the final type, a representative of the Ekedahl-Oort type"""
    
    unique_words = []
    multiplicity = []
    length = []
    unique_cycles = []
    
    #Gathers all the unique words that appear up to shifting of words
    #Also gathers the length and multiplicity of all these unique words
    letterCount = 0
    index_count = -1
    for w in W:
        for a in w:
            letterCount += 1
        if w not in unique_words:
            unique_words.append(w)
            multiplicity.append(1)
            length.append(len(w))
            index_count += 1
        else:
            index = unique_words.index(w)
            multiplicity[index] += 1
            
    #Need to make all words the same length by raising them to a power. This gives the smallest power that works for all words
    word_length = lcm(length)

    eoType = [-1 for i in range(letterCount/2)]
    
    #For all unique words that appear, give every possible cycle using those same letters and raises them to a power to get them to be the same length
    for i in range(len(unique_words)):
        cycles = [all_cycles(unique_words[i], j)*(int(word_length/length[i])) for j in range(length[i])]
        unique_cycles.append(cycles)

    #Sorts the words in lexicographical order with F<V
    sorted_words = sorted([w for words in unique_cycles for w in words], key=cmp_to_key(compare_words), reverse=True)
    
    #tau stores the dimension of the subspace in the canonical filtration corresponding to each word
    tau = []
    for i in range(len(unique_words)):
        for j in range(length[i]):
            for k in range(multiplicity[i]):
                w = unique_cycles[i][j]# * int(word_length/length[i])
                tau.append([sorted_words.index(w),i,j])
    
    #Final type is a non-decreasing list of non-negative integers of length g, where g is the genus of the curve.
    #The canonical type at the i-th spot is the dimension of Frobenius F applied to the subspace associated to the word with dimension i as a vector space
    dim = []
    for i in range(1,len(sorted_words)+1):
        n = 0
        basisList = []
        eoCount = 0
        for t in tau:
            if t[0] <= i-1:
                w = unique_words[t[1]]
                
                if w[length[t[1]] - t[2] - 1] == "F":
                    eoCount+=1
                        
                n += 1
        if n <= letterCount/2:
            eoType[n-1] = eoCount

    return eoType

In [12]:
def final_type(eoType):
    """Takes canonical type and fills in the blanks to return the final type"""
    last = 0
    thisRun = []

    #The canonical type has some 0's. This computes the final filtration and fills in the dimension of F applied to each subspace
    for i in range(len(eoType)):
        if eoType[i] == -1:
            thisRun.append(i)
        else:
            for j in range(len(thisRun)):
                if eoType[i] == last:
                    eoType[i-j-1] = last
                else:
                    eoType[i-j-1] = eoType[i] - j - 1
            last = eoType[i]
            thisRun = []

    return eoType

In [13]:
def eo_dimension(eoType):
    """Computes dimension of locus of A_g with given EO-type, which is the sum of the numbers in the final type."""
    summation = 0
    for a in eoType:
        summation += a
    return summation

In [14]:
def np_dimension(slopes):
    """Computes dimension of locus of A_g with given symmetric Newton polygon""" 
    #Dimension is number of lattice points above or on Newton polygon and below line y=x
    g = len(slopes)/2
    
    yVal = 0
    dim = 0
    for i in range(g):

        yVal += slopes[i]

        for j in range(ceil(yVal), i+1):
            if (j < i+1 and j <= g):
                dim += 1
            
    return dim
    

In [15]:
#there is probably a way to get the orbits using built-in methods, but it's also easy to do directly
def find_orbits(p,m):
    """Find the orbits of the multiplication by p map on Z/mZ - {0}"""
    G = Integers(m)
    used_elements = {G(0)} 
    orbits = {G(0)}
    orbits.remove(G(0)) #don't want the trivial orbit

    for g in G:
        if g not in used_elements:
            new_orbit = [g]
            current = p * g
            while current != g:
                new_orbit.append(current)
                used_elements.add(current)
                current = p * current
            orbits.add(tuple(new_orbit))

    return orbits

def compute_slopes(p,a1,a2,a3,m):
    """Computes the slopes, returing a list of the G_{c,d} appearing in the decomposition"""
    orbits = find_orbits(p,m)
    slopes = []
    lineSlopes = []

    for orbit in orbits:
        j = orbit[0]
        dj = compute_dj(j,m)
        if a1 % dj != 0 and a2 % dj != 0 and a3 % dj != 0:
            ones = sum([compute_fj(n.lift(),a1,a2,a3,m) for n in orbit])
            slopes.append((ones,len(orbit)-ones))
            for k in range(len(orbit)):
                lineSlopes.append(ones/len(orbit))
    
    return slopes,sorted(lineSlopes) 

def is_supersingular(p,a1,a2,a3,m):
    """Is the cover of degree m with inertial generators a1,a2,a3 supersingular in characteristic p?  Note depends only on p modulo m"""
    slopes = compute_slopes(p,a1,a2,a3,m)[0]

    for chunk in slopes:
        if chunk[1] != 0 and chunk[0]/chunk[1] != 1: #equivalent to slope 1/2
            return false

    return true

def is_ul_newton(p,a1,a2,a3,m, multiplier):
    """Does the Newton polygon of cover of degree m with inertial generators a1,a2,a3 represent an unlikely intersection in characteristic p?  Note depends only on p modulo m"""
    #multiplier is used when looking for 
    slopes = compute_slopes(p,a1,a2,a3,m)[1]
    moduli_dim = moduliDimension(slopes)
    g = len(slopes)/2
    
    #Codimension of locus of abelian varieties in A_g with given Newton polygon > multiplier*(dim of M_g) 
    if g*(g+1)/2 - moduli_dim > multiplier* (3*g-3):
        return true

    return false


In [16]:
def find_supersingular(a1,a2,a3,m):
    """Find all congruence class of primes modulo m (with gcd(p,m)=1) such that the cover is supersingular"""
    congruence_classes = dict()
    for p in range(m):
        if gcd(p,m) == 1:
            if is_supersingular(p,a1,a2,a3,m):
                congruence_classes[p] = 0
    return congruence_classes

def find_ul_newton(a1,a2,a3,m):
    """Find all congruence class of primes modulo m (with gcd(p,m)=1) such that the cover has an unlikely Newton polygon"""
    congruence_classes = dict()
    for p in range(m):
        if gcd(p,m) == 1:
            if is_ul_newton(p,a1,a2,a3,m):
                congruence_classes[p] = 0
    return congruence_classes

In [17]:
def computeType(c0,s,aList,mList,d,e):
    """Computes the signature type using the Chevalley-Weil formula"""
    sigType = -1
    for i in range(len(mList)):
        R = Integers(mList[i])
        term = 2*d*aList[i][0]+2*e*aList[i][1]*(c0/s)
        term = term/c0 * mList[i]
        term = Integer(R(term/2))
        fraction = -term/mList[i]
        sigType += fraction - floor(fraction)
    return sigType

In [18]:
def computeOrbits(c0,s,p):
    """Computes the orbits of the elements of Z/c0 \times Z/s by the action of multiplication by p"""
    Rc = Integers(c0)
    Rs = Integers(s)
    orbitList = []
    newOrbit = True
    for i in range(0,c0):
        for j in range(0,s):
            for o in orbitList:
                if [i,j] in o:
                    newOrbit = False
            if newOrbit:
                orbit = [[i,j]]
                notInOrbit = True
                
                while notInOrbit:
                    iNew = Integer(Rc(i*p))
                    jNew = Integer(Rs(j*p))
                    if [iNew, jNew] in orbit:
                        notInOrbit = False
                        break
                    orbit.append([iNew,jNew])
                    i = iNew
                    j = jNew
                orbitList.append(orbit)
            newOrbit = True
    return orbitList

In [19]:
def computeKernel(c0,s,a,b):
    """Computes the kernel of the function f: Z/c0 \times Z/s \to \mathbb{C} where f(x,y) = \zeta_(c_0)^(ax)*\zeta_s^(by), where \zeta_b is
    a primitive b-th root of unity."""
    kernel = set(())
    R = Integers(c0)
    for i in range(0,c0):
        for j in range(0,s):
            if R(a*i + b*j*c0/s) == 0:
                kernel.add((i,j))
    return kernel

In [20]:
def quotientInertiaType(a,character,c0,s,n):
    """Computes the inertia type of a curve quotiented by the kernel of a function as above"""
    newInertiaType = []
    R = Integers(n)
    for b in a:
        inertia = Integer(R(n/c0*(b[0]*character[0] + b[1]*character[1]*c0/s)))
        if inertia != 0:
            newInertiaType.append(inertia)
    return newInertiaType

In [34]:
def shimuraTaniyama(inertia,ramification,p,groupSize):
    """Given an abelian cover of P^1 branched at three points, computes the slopes of the Newton polygon using the Shimura-Taniyama formula"""
    characters = []
    slopes = []
    characterKernels = []

    #Grabs the characters with signature type 1
    for i in range(groupSize[0]):
        for j in range(groupSize[1]):
            if i != 0 or j != 0:
                sigType = computeType(groupSize[0],groupSize[1],inertia,ramification[:len(ramification)-1],i,j)
                if sigType == 1:
                    characters.append([i,j])
                    kernel = computeKernel(groupSize[0],groupSize[1],i,j)
                    characterKernels.append(kernel)

    orbits = computeOrbits(groupSize[0],groupSize[1],p)

    #For every orbit by the action of multiplication by p, we quotient the curve by the kernel of an element in the orbit.
    for orbit in orbits:
        slope = 0
        kernel = computeKernel(groupSize[0],groupSize[1],orbit[0][0],orbit[0][1])
        print(orbit)
        isGenus0 = True
        
        for i in range(len(characters)):
            if kernel.issubset(characterKernels[i]):
                isGenus0 = False
                break

        #If we don't have the trivial case, the quotient curve is a cyclic cover of P^1 branched at three points, so we can apply
        #the Shimura-Taniyama formula to compute the slopes
        if not isGenus0:

            
            qGroupSize = groupSize[0]*groupSize[1]/len(kernel)

            newInertia = quotientInertiaType(inertia,orbit[0],groupSize[0],groupSize[1],qGroupSize)
            R = Integers(qGroupSize)
            print(newInertia)
            
            if newInertia != []:
                addSig = True

                x = qGroupSize/groupSize[0]*orbit[0][0]
                y = qGroupSize/groupSize[1]*orbit[0][1]
                
                if x != 0 or y != 0:
                    order = Integer(lcm(gcd(x,y),qGroupSize)/gcd(x,y))
                else:
                    order = 1

                for i in newInertia:
                    print(i,order)
                    if order.divides(i):

                        addSig=False
                if addSig:
                    
                    for c in orbit:
                        if c in characters:
                            slope += 1
                    for i in range(len(orbit)):

                        slopes.append(slope/len(orbit))
                print()
        
    return p,sorted(slopes)

In [26]:
def _compute_moduli(moduli_dict):
    """Given the exponents of every galois group of a curve with genus g, and a list of primes, computes the natural density
    of the set of primes contained in the set of all primes. If the computation takes too long, it cuts it off and does an underestimate."""
    
    modulus_list = moduli_dict.keys()
    prime_occurence_list = []
    new_modulus_list = []

    #Since some examples are  too big to compute, we put the moduli with the most congruence classes first.
    for modulus in modulus_list:
        dictLength = len(moduli_dict[modulus])
        if dictLength > 0:
            prime_occurence_list.append(dictLength/modulus)
            new_modulus_list.append(modulus)
    sorted_modulus_list = [x for _, x in sorted(zip(prime_occurence_list, new_modulus_list), reverse=True)]
    
    if sorted_modulus_list == []:
        return 0
    
    m = sorted_modulus_list[0]
    new_modulus_list = [m]
    underestimate = False
    for modulus in sorted_modulus_list[1:]:
        
        new_lcm = lcm(m,modulus)
        #This value is somewhat arbitrary, we noticed that around this value for the lcm, the computation took much longer.
        if new_lcm <= 200000000:
            m = new_lcm
            new_modulus_list.append(modulus)
        else:
            underestimate = True
    if underestimate:
        print("Underestimate")
    num_moduli = 0
    
    print(m)
    
    for i in range(1,m):
        if gcd(i,m) == 1:
            for modulus in new_modulus_list:
                if i % modulus in moduli_dict[modulus]:
                    num_moduli += 1
                    break
    
    
    return num_moduli/euler_phi(m)    

def compute_densities(min,max):
    """Compute the density of primes for which we have supersingular 3-pointed covers with min <= genus <= max.
    Relies on non_cyclic_congruences[g] being non-empty (i.e. we worked out what happens when s != 1).
    If empty, just analyzes the cyclic examples."""

    collected_covers = present_by_genus(min,max)
    data_by_genus = []
    
    
    for g in range(min,max+1):
        print(g)
        moduli_dict = dict()

        #Loops through all covers of genus g.
        for data in collected_covers[g]:
            if data[3] == 1: #will be cyclic
                z = data # [a,b,c,s]
                possible_gens,m = cyclic_inertial_generators(*data)
                if not m in moduli_dict:
                    moduli_dict[m] = dict()
                for alist in possible_gens:
                    moduli_dict[m].update(find_supersingular(*alist,m))
            else:
                
                gen_list = alternate_inertial_generators(*data)
                for gen in gen_list:
                    groupSize = list(gen[0].invariants())
                    groupSize.reverse()
                    
                    if len(groupSize) == 1:
                        groupSize.append(1)
                    
                    groupOrder = groupSize[0]*groupSize[1]
                    
                    if not groupSize[0] in moduli_dict:
                        moduli_dict[groupSize[0]] = dict()

                    #Grabs the slopes for all covers and records whether or not it is supersingular
                    for i in range(1,len(gen)):
                        for j in range(1,groupSize[0]):
                            if gcd(j,groupSize[0]) == 1:
                                slopeData = shimuraTaniyama(gen[i],data,j,groupSize)
                                prime = slopeData[0]
                                slopes = slopeData[1]
                                
                                supersingular = True
                                for slope in slopes:
                                    if slope != 1/2:
                                        supersingular = False
                                        break
                               
                                if supersingular:
                                    moduli_dict[groupSize[0]].update({prime: 0})
        print("Analyzing density")
        density = _compute_moduli(moduli_dict)

        data_by_genus.append(density)
        print(g,density)
        print("")
    return data_by_genus

In [27]:
def cycle_word(w):
    """Given a word in the characters F and V, returns all cycles of that word"""   
    a = w
    if a[0] == "F" and a[-1] == "V":
        return a
    for i in range(1,len(w)):
        a = w[i:] + w[:i]
        
        if a[0] == "F" and a[-1] == "V":
            return a

In [28]:
def ekedahl_oort_type(inertia, ramification, p, groupSize):
    """In a similar vein to the Shimura-Taniyama function, this computes the Ekedahl-Oort type by taking quotients of the curve"""
    characters = []
    words = []
    characterKernels = []
    
    for i in range(groupSize[0]):
        for j in range(groupSize[1]):
            if i != 0 or j != 0:
                sigType = computeType(groupSize[0],groupSize[1],inertia,ramification[:len(ramification)-1],i,j)
                if sigType == 1:
                    characters.append([i,j])
                    kernel = computeKernel(groupSize[0],groupSize[1],i,j)
                    characterKernels.append(kernel)

    orbits = computeOrbits(groupSize[0],groupSize[1],p)
    
    for orbit in orbits:
        
        slope = 0
        kernel = computeKernel(groupSize[0],groupSize[1],orbit[0][0],orbit[0][1])

        isGenus0 = True
        
        for i in range(len(characters)):
            if kernel.issubset(characterKernels[i]):
                isGenus0 = False
                break
        
        if not isGenus0:

            word = ""
            fAppears = False
            vAppears = False
            for character in orbit:
                conjugate = [(groupSize[0] - character[0]) % groupSize[0], (groupSize[1] - character[1]) % groupSize[1]]
                if conjugate in characters:
                    word += "F"
                    fAppears = True
                else:
                    word += "V"
                    vAppears = True
            if fAppears and vAppears:
                words.append(cycle_word(word))
            else:
                words.append(word)
    return words

In [30]:
def superspecial(words):
    """Returns true if the only words given by a curve are 'FV'"""
    for word in words:
        if word != "FV":
            return False
    return True

In [31]:
def eo_prime_density(min, max):
    """Computes the density of primes such that there is a superspecial curve per genus, with genus in between min and max"""
    collected_covers = present_by_genus(min,max)
    data_by_genus = []
    
    
    for g in range(min,max+1):
        print(g)
        moduli_dict = dict()
        
        for data in collected_covers[g]:
            if data[3] == 1: #will be cyclic
                z = data # [a,b,c,s]
                possible_gens,m = cyclic_inertial_generators(*data)
                groupSize = [m,1]

                new_gens = []
                for gens in possible_gens:
                    new_gens.append([[gens[0],1],[gens[1],1],[gens[2],1]])
                if not m in moduli_dict:
                    moduli_dict[m] = dict()
                for gen in new_gens:
                    for i in range(1,m):
                        if gcd(i,m) == 1:
                            eoType = ekedahl_oort_type(gen,data,i,groupSize)
                            if superspecial(eoType):
                                if not i in moduli_dict[m]:
                                    moduli_dict[m][i] = dict()
                                
                                moduli_dict[m][i] = 0
            else:
                
                gen_list = alternate_inertial_generators(*data)
                for gen in gen_list:
                    groupSize = list(gen[0].invariants())
                    groupSize.reverse()
                    
                    if len(groupSize) == 1:
                        groupSize.append(1)
                    
                    groupOrder = groupSize[0]*groupSize[1]
                    
                    if not groupSize[0] in moduli_dict:
                        moduli_dict[groupSize[0]] = dict()
                    
                    for i in range(1,len(gen)):
                        for j in range(1,groupSize[0]):
                            if gcd(j,groupSize[0]) == 1:
                                
                                eoType = ekedahl_oort_type(gen[i],data,j,groupSize)
                                if superspecial(eoType):
                                    if not j in moduli_dict[groupSize[0]]:
                                        moduli_dict[groupSize[0]][j] = dict()
                                
                                    moduli_dict[groupSize[0]][j] = 0
        print("Analyzing density")

        density = _compute_moduli(moduli_dict)

        data_by_genus.append(density)
        print(g,density)
        print("")
    return data_by_genus

In [32]:
def ul_newton_prime_density(min, max):
    """Computes the density of primes such that the covers have an unlikely Newton polygon per genus, with genus in between min and max"""
    collected_covers = present_by_genus(min,max)
    data_by_genus = []
    
    
    for g in range(min,max+1):
        print(g)
        moduli_dict = dict()
        
        for data in collected_covers[g]:
            if data[3] == 1: #will be cyclic
                z = data # [a,b,c,s]
                possible_gens,m = cyclic_inertial_generators(*data)
                if not m in moduli_dict:
                    moduli_dict[m] = dict()
                for alist in possible_gens:
                    moduli_dict[m].update(find_ul_newton(*alist,m))
            else:
                
                gen_list = alternate_inertial_generators(*data)
                for gen in gen_list:
                    groupSize = list(gen[0].invariants())
                    groupSize.reverse()
                    
                    if len(groupSize) == 1:
                        groupSize.append(1)
                    
                    groupOrder = groupSize[0]*groupSize[1]
                    
                    if not groupSize[0] in moduli_dict:
                        moduli_dict[groupSize[0]] = dict()
                    
                    for i in range(1,len(gen)):
                        for j in range(1,groupSize[0]):
                            if gcd(j,groupSize[0]) == 1:
                                slopeData = shimuraTaniyama(gen[i],data,j,groupSize)
                                #print(slopeData)
                                prime = slopeData[0]
                                modDim = moduliDimension(slopeData[1])
                                if g*(g+1)/2 - modDim > 2*(3*g-3):
                                    moduli_dict[groupSize[0]].update({prime:0})

        print("Analyzing density")
        density = _compute_moduli(moduli_dict)
        #print(combined)
        #print(common_modulus)

        data_by_genus.append(density)
        print(g,density)
        print("")
    return data_by_genus

In [33]:
def eo_ul_density(min, max):
    """Computes the density of primes such that the covers have an unlikely Ekedahl-Oort type per genus, with genus in between min and max"""
    collected_covers = present_by_genus(min,max)
    data_by_genus = []
    
    
    for g in range(min,max+1):
        print(g)
        moduli_dict = dict()
        
        for data in collected_covers[g]:
          
            if data[3] == 1: #will be cyclic
                z = data # [a,b,c,s]
                possible_gens,m = cyclic_inertial_generators(*data)
                groupSize = [m,1]

                new_gens = []
                for gens in possible_gens:
                    new_gens.append([[gens[0],1],[gens[1],1],[gens[2],1]])
                if not m in moduli_dict:
                    moduli_dict[m] = dict()
                for gen in new_gens:
                 
                    for i in range(1,m):
                        if gcd(i,m) == 1:
                            words = ekedahl_oort_type(gen,data,i,groupSize)


                            eoType = final_type(canonical_type(words))
                            
                            if g*(g+1)/2 - eo_dimension(eoType) > 2*(3*g-3):
                            
                                if not i in moduli_dict[m]:
                                    moduli_dict[m][i] = dict()
                                moduli_dict[m][i] = 0

                                
            else:
                
                gen_list = alternate_inertial_generators(*data)
                for gen in gen_list:
                    groupSize = list(gen[0].invariants())
                    groupSize.reverse()
                    
                    if len(groupSize) == 1:
                        groupSize.append(1)
                    
                    groupOrder = groupSize[0]*groupSize[1]
                    
                    if not groupSize[0] in moduli_dict:
                        moduli_dict[groupSize[0]] = dict()
                    
                    for i in range(1,len(gen)):

                        for j in range(1,groupSize[0]):
                            
                            if gcd(j,groupSize[0]) == 1:
                                
                                words = ekedahl_oort_type(gen[i],data,j,groupSize)
                                
                                
                                eoType = final_type(canonical_type(words))
                                
                                if g*(g+1)/2 - eo_dimension(eoType) > 2*(3*g-3):

                                    if not j in moduli_dict[groupSize[0]]:
                                        moduli_dict[groupSize[0]][j] = dict()
                                    moduli_dict[groupSize[0]][j] = 0
                                

        print("Analyzing density")

        density = _compute_moduli(moduli_dict)


        data_by_genus.append(density)
        print(g,density)
        print("")
    return data_by_genus